In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [3]:
random_state = 0

# IMPORT DATASET

In [4]:
dataset_path = parentdir + "/datasets/HARDataset/"

In [5]:
dataset_name = "HARDataset"

In [6]:
X = pd.read_csv(dataset_path + "/train/" + "X_train.txt", header=None, delim_whitespace=True)

X = pd.DataFrame(X.values)

y = pd.read_csv(dataset_path + "/train/" + "y_train.txt", header = None, delim_whitespace=True).values
y_all = np.ravel(y).astype("int")
le = LabelEncoder()
le.fit(y_all)
y_all = le.transform(y_all)
#X["class"] = y
X_all = X.values.reshape((X.shape[0], X.shape[1], 1))

In [7]:
print("X SHAPE: ", X_all.shape)
print("y SHAPE: ", y_all.shape)
unique, counts = np.unique(y_all, return_counts=True)
print("\nCLASSES BALANCE")
for i, label in enumerate(unique):
    print(label, ": ", round(counts[i]/sum(counts), 2))

X SHAPE:  (7352, 561, 1)
y SHAPE:  (7352,)

CLASSES BALANCE
0 :  0.17
1 :  0.15
2 :  0.13
3 :  0.17
4 :  0.19
5 :  0.19


In [8]:
X_test = pd.read_csv(dataset_path + "/test/" + "X_test.txt", header=None, delim_whitespace=True)

X_test = pd.DataFrame(X_test.values)

y_test = pd.read_csv(dataset_path + "/test/" + "y_test.txt", header = None, delim_whitespace=True).values
y_test = np.ravel(y_test).astype("int")
y_test = le.transform(y_test)
#X["class"] = y
X_test = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [9]:
print("X SHAPE: ", X_test.shape)
print("y SHAPE: ", y_test.shape)

X SHAPE:  (2947, 561, 1)
y SHAPE:  (2947,)


## DATASET SPLIT

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# BLACKBOX/EXPLANATION SETS SPLIT
X_train, X_exp, y_train, y_exp = train_test_split(X_all, y_all, 
                                                  test_size=0.3, stratify = y_all, random_state=random_state)

# BLACKBOX TRAIN/VALIDATION SETS SPLIT
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, stratify = y_train, random_state=random_state)

# EXPLANATION TRAIN/TEST SETS SPLIT
X_exp_train, X_exp_test, y_exp_train, y_exp_test = train_test_split(X_exp, y_exp, 
                                                                    test_size=0.2, 
                                                                    stratify = y_exp, 
                                                                    random_state=random_state)

# EXPLANATION TRAIN/VALIDATION SETS SPLIT
X_exp_train, X_exp_val, y_exp_train, y_exp_val = train_test_split(X_exp_train, y_exp_train, 
                                                                  test_size=0.2, 
                                                                  stratify = y_exp_train, 
                                                                  random_state=random_state)

print("SHAPES:")
print("BLACKBOX TRAINING SET: ", X_train.shape)
print("BLACKBOX VALIDATION SET: ", X_val.shape)
print("BLACKBOX TEST SET: ", X_test.shape)
print("EXPLANATION TRAINING SET: ", X_exp_train.shape)
print("EXPLANATION VALIDATION SET: ", X_exp_val.shape)
print("EXPLANATION TEST SET: ", X_exp_test.shape)

SHAPES:
BLACKBOX TRAINING SET:  (4116, 561, 1)
BLACKBOX VALIDATION SET:  (1030, 561, 1)
BLACKBOX TEST SET:  (2947, 561, 1)
EXPLANATION TRAINING SET:  (1411, 561, 1)
EXPLANATION VALIDATION SET:  (353, 561, 1)
EXPLANATION TEST SET:  (442, 561, 1)


In [12]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_all)), 1 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)

TIMESTEPS:  561
N. LABELS:  6


# BLACKBOX MODELS

In [13]:
import keras
from sklearn.metrics import accuracy_score
import time
from blackboxes import *
import blackboxes
from myutils import reconstruction_blackbox_consistency

Using TensorFlow backend.
/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


## resnet

In [14]:
blackbox = build_resnet(n_timesteps, n_outputs)
blackbox.load_weights(parentdir + "/blackbox_checkpoints/HARDataset_blackbox_resnet_20191028_172136_best_weights_+0.99_.hdf5")
resnet = blackbox

## simple CNN

In [15]:
blackbox = build_simple_CNN(n_timesteps, n_outputs)
blackbox.load_weights(parentdir + "/blackbox_checkpoints/HARDataset_blackbox_simpleCNN_20191029_153407_best_weights_+0.94_.hdf5")
simplecnn = blackbox

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## KNN

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, mean_squared_error, accuracy_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [17]:
knn = load(parentdir + "/blackbox_checkpoints/HARDataset_blackbox_knn_20191031_111540.joblib")
#knn.score(X_test.reshape(X_test.shape[:2]), y_test)
#mean_squared_error(y_test, knn.predict(X_test.reshape(X_test.shape[:2])))

# AUTOENCODER

In [18]:
import autoencoders
from autoencoders import *
"""
import importlib
importlib.reload(autoencoders)
"""

'\nimport importlib\nimportlib.reload(autoencoders)\n'

## Standard

In [19]:
params = {"input_shape": (n_timesteps,1),
          "n_blocks": 8, 
          "latent_dim": 50,
          "encoder_latent_layer_type": "dense",
          "encoder_args": {"filters":[2,4,8,16,32,64,128,256], 
                            "kernel_size":[21,18,15,13,11,8,5,3], 
                            "padding":"same", 
                            "activation":"selu", 
                            "pooling":[1,1,1,1,1,1,1,1]}
         }

aut = Autoencoder(verbose = False, **params)
encoder, decoder, autoencoder = aut.build()
autoencoder.load_weights(parentdir + "/autoencoder_checkpoints/HARDataset_autoencoder_20191031_212226_best_weights_+0.008519_.hdf5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



# GLOBAL VS LOCAL SHAPELET TREE

In [20]:
# 50 test set
_, X_exp_test_50, _, y_exp_test_50 = train_test_split(X_exp_test, y_exp_test, 
                                                                  test_size=50, 
                                                                  stratify = y_exp_test, 
                                                                  random_state=random_state)

In [74]:
import agnosticlocalexplainer
from agnosticlocalexplainer import *

from myutils import BlackboxPredictWrapper
import time
from agnosticglobalexplainer import AgnosticGlobalExplainer, save_shapelet_model, load_shapelet_model
from joblib import dump
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, coverage_error
from global_vs_local_surrogate import build_agnostic_local_explainers, print_report, massive_save_agnostic_local_explainers, massive_load_agnostic_local_explainers, get_all_predictions

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    4.3s finished


## RESNET

In [ ]:
autoencoder = autoencoder
encoder = autoencoder.layers[1]
decoder = autoencoder.layers[2]
blackbox = resnet
blackbox_input_dimensions = 3
labels = ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"]

In [ ]:
max_iter = 50
file_path = parentdir + "/agnostic_explainers/" + dataset_name + "_" + time.strftime("%Y%m%d_%H%M%S")

In [ ]:
import platform
if platform.system() == "Windows":
    os.makedirs(file_path + "/")
else: os.mkdir(file_path + "/")

In [ ]:
%%time
global_surrogate = AgnosticGlobalExplainer(random_state = random_state, 
                                           max_iter = max_iter, 
                                           distance_quantile_threshold = np.array(list(range(1,10)))/10,
                                           labels = labels)
global_surrogate.fit(X_exp_train[:,:,0], blackbox_predict.predict(X_exp_train))

In [22]:
%%time
agnostic_explainers = build_agnostic_local_explainers(blackbox, 
                               encoder, 
                               decoder, 
                               autoencoder, 
                               X_exp_test_50, 
                               y_exp_test_50,
                               blackbox_input_dimensions = blackbox_input_dimensions,
                               labels = labels,
                               size = 1000,
                               neigh_type = "geneticp",
                               ngen = 10,
                              max_iter=max_iter,
                              random_state = random_state,
                              distance_quantile_threshold = np.array(list(range(1,10)))/10
                               )


Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	421   	0.669083	0.472968	0.991709
2  	428   	0.81328 	0.449427	0.993249
3  	427   	0.847267	0.430785	0.993249
4  	424   	0.849115	0.466609	0.993249
5  	422   	0.84671 	0.44492 	0.993249
6  	428   	0.862477	0.4435  	0.993249
7  	425   	0.856708	0.449603	0.993249
8  	437   	0.877868	0.465643	0.993249
9  	428   	0.883182	0.464191	0.993193
10 	430   	0.879285	0.456051	0.993183
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	426   	0.574413	0.480808	0.9909
2  	438   	0.718152	0.468315	0.9909
3  	428   	0.834665	0.461259	0.9909
4  	429   	0.897798	0.471617	0.9909
5  	410   	0.931205	0.472769	0.9909
6  	432   	0.929923	0.466421	0.9909
7  	433   	0.933424	0.469467	0.9909
8  	425   	0.919441	0.473461	0.9905
9  	427   	0.922651	0.479977	0.990951
10 	412   	0.929212	0.466777	0.99116 
synthetic neighborhood class counts {0: 470, 1: 476, 2: 22, 3: 87, 4: 41, 5: 4}
Fi

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 175 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    4.4s finished


1 / 50

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	421   	0.652574	0.475217	0.992212
2  	428   	0.808018	0.470732	0.993294
3  	427   	0.830732	0.448629	0.993267
4  	424   	0.843142	0.44248 	0.993267
5  	422   	0.855822	0.450094	0.993267
6  	428   	0.878275	0.470551	0.993267
7  	425   	0.853487	0.477187	0.993267
8  	437   	0.881006	0.477491	0.993269
9  	428   	0.865332	0.469951	0.993269
10 	430   	0.853702	0.475065	0.993269
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	426   	0.59882	0.486762	0.991334
2  	438   	0.752208	0.477534	0.991806
3  	428   	0.872902	0.480879	0.991806
4  	429   	0.913982	0.484819	0.991688
5  	410   	0.921238	0.484572	0.992089
6  	432   	0.91292 	0.474554	0.992058
7  	433   	0.897031	0.477616	0.992157
8  	425   	0.918553	0.48142 	0.992159
9  	427   	0.917104	0.473177	0.992157
10 	412   	0.926787	0.483091	0.992318
synthetic neighborhood class counts {0: 51, 1: 86, 2: 37, 3:

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    3.9s finished


2 / 50

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	421   	0.693917	0.484118	0.992217
2  	428   	0.838006	0.472739	0.993303
3  	427   	0.892749	0.483673	0.993262
4  	424   	0.897543	0.477671	0.993262
5  	422   	0.892221	0.483144	0.993262
6  	428   	0.906993	0.485524	0.993262
7  	425   	0.907487	0.483856	0.993262
8  	437   	0.910924	0.487458	0.993262
9  	428   	0.898161	0.481486	0.993112
10 	430   	0.909887	0.483109	0.993181
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	426   	0.555569	0.490063	0.990919
2  	438   	0.674979	0.488139	0.991668
3  	428   	0.810769	0.486904	0.991668
4  	429   	0.87919 	0.488095	0.991693
5  	410   	0.896453	0.489851	0.991693
6  	432   	0.897446	0.487756	0.991693
7  	433   	0.901882	0.49026 	0.991693
8  	425   	0.901725	0.489672	0.991693
9  	427   	0.88502 	0.487536	0.991693
10 	412   	0.873131	0.48869 	0.991693
synthetic neighborhood class counts {0: 65, 1: 30, 2: 19, 3

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    2.2s finished


49 / 50

Neighborhood Generation
calculating feature values
gen	nevals	avg     	min     	max     
0  	500   	0.496654	0.496654	0.496654
1  	421   	0.681662	0.476014	0.991797
2  	428   	0.828009	0.41647 	0.993292
3  	427   	0.885978	0.437028	0.993263
4  	424   	0.8832  	0.455075	0.993263
5  	422   	0.899275	0.446909	0.993263
6  	428   	0.887425	0.453614	0.993263
7  	425   	0.897003	0.461511	0.993263
8  	437   	0.896338	0.460931	0.993263
9  	428   	0.904991	0.442198	0.993202
10 	430   	0.905231	0.459242	0.993202
gen	nevals	avg	min	max
0  	500   	0.5	0.5	0.5
1  	426   	0.557398	0.47899	0.990198
2  	438   	0.677802	0.445355	0.99189 
3  	428   	0.809618	0.465246	0.99189 
4  	429   	0.870902	0.436454	0.99189 
5  	410   	0.88222 	0.467926	0.99189 
6  	432   	0.900432	0.463197	0.99189 
7  	433   	0.893085	0.471529	0.991921
8  	425   	0.891644	0.454949	0.99189 
9  	427   	0.906725	0.45344 	0.99189 
10 	412   	0.876417	0.458118	0.99189 
synthetic neighborhood class counts {0: 563, 1: 364, 2: 103

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


50 / 50
CPU times: user 17h 42min 14s, sys: 1h 48min 13s, total: 19h 30min 27s
Wall time: 10h 3min 32s


[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    2.2s finished


In [75]:
results_df = get_all_predictions(agnostic_explainers, global_surrogate, blackbox_predict, X_exp_test_50, blackbox_predict.predict(X_exp_train))
results_df.to_csv(file_path + "/" + "results_df.csv", sep = ";", index = False)
print_report(results_df)

local fidelity:  0.98
global fidelity:  0.56
reconstruction fidelity:  0.98


local coverage:  1.0
global coverage:  2.0


local classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         7
           3       1.00      0.89      0.94         9
           4       0.90      1.00      0.95         9
           5       1.00      1.00      1.00        10

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50



global classification report:
              precision    recall  f1-score   support

           0       0.57      0.50      0.53         8
           1       0.60      0.43      0.50         7
           2       0.56      0.71      0.63         7
           3       0.43      0.33      0.38         9
  

In [77]:
%%time
save_shapelet_model(global_surrogate, file_path + "/")
massive_save_agnostic_local_explainers(agnostic_explainers, file_path, verbose = True)

['/Users/francesco/github/TS_AgnosticLocalExplainer/agnostic_explainers/HARDataset_agnosticvsglobal_20191211_181641.joblib']

In [84]:
%%time
global_surrogate = load_shapelet_model(file_path + "/")
agnostic_explainers = massive_load_agnostic_local_explainers(file_path, verbose = True)

['/Users/francesco/github/TS_AgnosticLocalExplainer/agnostic_explainers/HARDataset_agnosticvsglobal_20191211_181641_results_y.joblib']

In [78]:
# sanity check
results_df_loaded = get_all_predictions(agnostic_explainers, global_surrogate, blackbox_predict)
print(sum(results_df_loaded.values != results_df.values))

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    2.9s finished
